# DEM of a Stack of Blocks

## Add blocks to a model

First we create a base block geometry using a simple box.

In [1]:
from compas.geometry import Box

box = Box(xsize=1, ysize=1, zsize=1)

We will add 10 blocks to a block model using the box as base geometry.
In addition, we will add a random rotation and a random translation to each block to create an irregular stack.

In [2]:
import math
import random

from compas.geometry import Rotation
from compas.geometry import Translation

from compas_dem.elements import BlockElement
from compas_dem.models import BlockModel

model = BlockModel()

for i in range(10):
    T = Translation.from_vector([
        random.choice([-0.1, 0.1]) * random.random(),
        random.choice([-0.1, 0.1]) * random.random(),
        i * box.zsize,
    ])
    R = Rotation.from_axis_and_angle(
        axis=[0, 0, 1],
        angle=math.radians(random.choice([-5, +5])),
    )

    block = BlockElement.from_box(box)
    block.transformation = T * R

    model.add_element(block)

## Compute the interfaces between the blocks

In a stack like this, each block can have at most two interfaces.
Therefore, we limit the number of neighbours that should be considered for calculating contact interfaces for each block to two (`k=2`).

In [3]:
model.compute_contacts(k=2)

## Identify the Support

We identify the support block as the block with the reference point (i.e. the centroid of its geometry) with the lowest z-coordinate.

In [4]:
block: BlockElement = sorted(model.elements(), key=lambda e: e.point.z)[0]
block.is_support = True

## Visualize the result

In [ ]:
from compas.colors import Color
from compas_notebook import Viewer

viewer = Viewer()

for element in model.elements():
    viewer.scene.add(
        element.modelgeometry,
        show_faces=True if element.is_support else False,
        show_edges=True,
        facecolor=Color.red() if element.is_support else None,
        edgecolor=Color.black()
    )

for contact in model.contacts():
    viewer.scene.add(contact.polygon, color=Color.cyan())

viewer.show()

viewer.camera3.position = [-10, -10, 7]
viewer.camera3.lookAt([0, 0, 5])
